# Get Climate data

## Bulk-load RDF data to Neptune

In [ ]:
import os
import subprocess

stream = os.popen("source ~/.bashrc ; echo $DATA_BUCKET")
lines=stream.read().split("\n")
DATA_BUCKET=lines[0]
DATA_BUCKET
%load -s s3://{DATA_BUCKET}/rdf -f nquads --store-to loadres1 --run


In [ ]:
%load_status {loadres1['payload']['loadId']} --errors --details


## Run queries across graph and lake!

In each query, copy the private IP of ECS task.

In [ ]:
%%sparql

select * where {
    <http://climate.aws.com/resource#3129099999> ?p ?o}


In [ ]:
%%sparql
PREFIX clmo: <http://climate.aws.com/ontology/>
PREFIX clmr: <http://climate.aws.com/resource#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>


# readings at Manchester UK in 
select * where {
    BIND(clmr:3334099999 as ?stat) .
    ?stat rdfs:label ?name .
    ?stat clmo:latitude ?lat .
    ?stat clmo:longitude ?long .
    
    SERVICE <http://172.30.1.4:8080/sparql> {
       ?reading clmo:station ?stat ;
            clmo:celsius ?celsius ;
            clmo:dateTime ?dateTime ;
            clmo:fahrenheit ?fahrenheit ;
            #FILTER(?dateTime > "1985-12-04T13:20:00"^^xsd:dateTime) .
            FILTER(?celsius >= "30"^^xsd:double) .
    } 
} 
LIMIT 10


In [ ]:
%%sparql
PREFIX clmo: <http://climate.aws.com/ontology/>
PREFIX clmr: <http://climate.aws.com/resource#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>


# readings at Manchester UK in 
select * where {
    BIND(clmr:3334099999 as ?stat) .
    ?stat rdfs:label ?name .
    ?stat clmo:latitude ?lat .
    ?stat clmo:longitude ?long .
    
    SERVICE <http://172.30.1.4:8080/sparql> {
       ?reading clmo:station ?stat ;
            clmo:celsius ?celsius ;
            clmo:dateTime ?dateTime ;
            clmo:fahrenheit ?fahrenheit ;
            FILTER(?dateTime > "1985-12-04T13:20:00"^^xsd:dateTime) .
            #FILTER(?celsius >= "30"^^xsd:double) .
    } 
} 
LIMIT 40